# Importing Libraries

In [1]:
import wandb
wandb.login(key='8ab0846d3cb35c36ca9a932bc0881dd0cc63df50')

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
import numpy as np
import pandas as pd
import json
import tensorflow as tf
import torch
import torch.optim as optim
import os
import cv2
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, Dataset
import random
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision
import torchvision.transforms.functional as fn
from tqdm.notebook import trange, tqdm, tqdm_notebook
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [3]:
class obj: 
    # constructor
    def __init__(self, dict1):
        self.__dict__.update(dict1)
   


In [4]:
def dict2obj(dict1):
      
    # using json.loads method and passing json.dumps
    # method and custom object hook as arguments
    return json.loads(json.dumps(dict1), object_hook=obj)

# Dataset Classes

In [5]:
def data_preload(path, shape, norm, reshape):
    data_classes = os.listdir(path)
    label = 0
    data_x = []
    data_y = []
    label = 0
    min_examples = min(list(map(lambda x: len(os.listdir(path+x+'/')), data_classes)))
    for d_class in data_classes:
        image_path = path+d_class+'/'
        image_names = os.listdir(image_path)
#         image_names = image_names[:min_examples]
        if reshape:
            image_data = np.array(list(map(lambda x:cv2.
                                     resize(cv2.
                                            imread(image_path+x),
                                            shape
                                           ).transpose(2,0 , 1)/norm, image_names)))
        else:
            image_data = np.array(list(map(lambda x:cv2.
                                     cv2.
                                    imread(image_path+x)
                                    .transpose(2,0 , 1)/norm, image_names)))
        data_x.append(image_data)
#     data_x = np.array(data_x)
    return data_x

In [6]:
# For OmniGlot
def data_preload_deep(path, shape, norm, reshape):
    data_classes = os.listdir(path)
    label = 0
    data_x = []
    data_y = []
    label = 0
    min_examples = min(
                    list(
                        map(
                            lambda x: min(
                                        list(
                                            map(
                                                lambda y: len(os.listdir(path+x+'/'+y)), 
                                                os.listdir(path+x+'/')
                                            )
                                            )
                                        ), data_classes)))
    for d_class in data_classes:
        sub_classes = os.listdir(path+d_class+'/')
        for sub in sub_classes:
            image_path = path+d_class+'/'+sub+'/'
            image_names = os.listdir(image_path)
#             image_names = image_names[:min_examples]
            if reshape:
                image_data = np.array(list(map(lambda x:cv2.
                                         resize(cv2.
                                                imread(image_path+x),
                                                shape
                                               ).transpose(2,0 , 1)/norm, image_names)))
            else:
                image_data = np.array(list(map(lambda x:cv2.
                                         cv2.
                                        imread(image_path+x)
                                        .transpose(2,0 , 1)/norm, image_names)))
            data_x.append(image_data)
#     data_x = np.array(data_x)
    return data_x

In [7]:
def print_data(data, name):
    print(name, end=' : ')
    for dat in data:
        print(dat.shape, end=' ')
    print('\n')

In [8]:
class ProtoDataset(Dataset):
    def __init__(self, data, Nc, Ns, Nq, epoch):
        self.nc = Nc
        self.ns = Ns
        self.nq = Nq
        self.epoch = epoch
        self.data = data
    def __len__(self):
        return self.epoch
    def __getitem__(self, i):
#         data_classes = list(range(self.data.shape[0]))
        data_classes = list(range(len(self.data)))
        select_classes = random.sample(data_classes, self.nc)
        query_images = []
        query_labels = []
        support_images = []
        support_labels = []
        for selected in select_classes:
            select_images_index = random.sample(
                                        list(
                                            range(
                                                self.data[selected].shape[0]
                                        )), self.nq+self.ns)
            select_query_images = list(
                                map(
                                    lambda x: self.data[selected][x, :, :, :],
                                    select_images_index[:self.nq]
                                ))
            select_support_images = list(
                                map(
                                    lambda x: self.data[selected][x, :, :, :],
                                    select_images_index[self.nq:]
                                ))
            select_query_labels = [selected]*self.nq
            select_support_labels = [selected]*self.ns
            query_images += select_query_images
            query_labels += select_query_labels
            support_images += select_support_images
            support_labels += select_support_labels
        query_images = torch.tensor(query_images).float().to(device)
        support_images = torch.tensor(support_images).float().to(device)
        query_labels = torch.tensor(query_labels).float().to(device)
        support_labels = torch.tensor(support_labels).float().to(device)
        return query_images, query_labels, support_images, support_labels

In [9]:
class DAPNDataset(Dataset):
    def __init__(self, data, Nc, Ns, Nq, Nv=1, valid_split=3, epoch=1):
        self.nc = Nc
        self.ns = Ns
        self.nq = Nq
        self.nv = Nv
        self.valid_split = valid_split
        self.epoch = epoch
        self.data = data
    def __len__(self):
        return self.epoch
    def __getitem__(self, i):
#         data_classes = list(range(self.data.shape[0]))
        all_classes = list(range(len(self.data)))
        validation_classes = random.sample(all_classes, self.valid_split)
        s = set(validation_classes)
        data_classes = [x for x in all_classes if x not in s]
        select_classes = random.sample(data_classes, self.nc)
        query_images = []
        query_labels = []
        support_images = []
        support_labels = []
        validation_images = []
        validation_labels = []
        for selected in validation_classes:
            select_images_index = random.sample(
                                        list(
                                            range(
                                                self.data[selected].shape[0]
                                        )), self.nv)
            select_validation_images = list(
                                map(
                                    lambda x: self.data[selected][x, :, :, :],
                                    select_images_index[:]
                                ))
            select_validation_labels = [selected]*self.nv
            validation_images += select_validation_images
            validation_labels += select_validation_labels
            
            
        for selected in select_classes:
            select_images_index = random.sample(
                                        list(
                                            range(
                                                self.data[selected].shape[0]
                                        )), self.nq+self.ns)
            select_query_images = list(
                                map(
                                    lambda x: self.data[selected][x, :, :, :],
                                    select_images_index[:self.nq]
                                ))
            select_support_images = list(
                                map(
                                    lambda x: self.data[selected][x, :, :, :],
                                    select_images_index[self.nq:]
                                ))
            select_query_labels = [selected]*self.nq
            select_support_labels = [selected]*self.ns
            query_images += select_query_images
            query_labels += select_query_labels
            support_images += select_support_images
            support_labels += select_support_labels
            
        query_images = torch.tensor(query_images).float().to(device)
        support_images = torch.tensor(support_images).float().to(device)
        validation_images = torch.tensor(validation_images).float().to(device)
        query_labels = torch.tensor(query_labels).float().to(device)
        support_labels = torch.tensor(support_labels).float().to(device)
        validation_labels = torch.tensor(validation_labels).float().to(device)
        
        return query_images, query_labels, support_images, support_labels, validation_images, validation_labels

# Model

In [10]:
class Encoder(nn.Module):
    def __init__(self, out_channels, blocks, name='0', drop=False):
        super(Encoder, self).__init__()
        self.name = name
        self.blocks = blocks
        self.drop = drop
        self.block0 = nn.Sequential(
            nn.Conv2d(3, out_channels, kernel_size=3,padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.block = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, kernel_size=3,padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        
        self.blockd = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, kernel_size=3,padding=1),
            nn.Dropout(p=0.1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
        )
    def forward(self, x):
        x1 = self.block0(x)
        if self.drop:
            for i in range(self.blocks-1):
                x1 = self.blockd(x1)
        else:
            for i in range(self.blocks-1):
                x1 = self.block(x1)
        return x1.reshape(x1.shape[0], -1)

# Loss, Prototype and Probability

In [11]:
def dist_metric(data, prototype, c=2, norm_dist=1):
    '''
    data: (d0, XX)
    prototype: (XX)
    returns : (d0)
    '''
    res = torch.pow(torch.pow(torch.abs(data-prototype), c).sum(1)/norm_dist, 1/c).to(device)
#     print('dist ', res)
    return res

In [12]:
def cluster_Prototype(x, y, c, norm_dist):
    '''
    c represents type of norm
    x (expected dimension) : (Ns*Nc, XX)
    y (expected dimension) : (Ns*Nc)
    indexes is a list of all encodings in x, where x has label 'unique_label'
    class_encodings further selects all such encodings from x from based on indexes from y
    class_encodings: (Ns, XX) x11 x22 ...............x1d
                              x21 x22 ...............x2d
                              .                        .
                              .                        .
                              .                        .
                              xn1 xn2 ...............xnd
    prototype : (XX)          x1 x2 x3 .................xd
    '''
    unique_labels = torch.unique(y)
    prototypes = []
    for unique_label in unique_labels:
        indexes = [i for i in range(y.shape[0]) if y[i]==unique_label]
        indexes_tensor = torch.tensor(indexes).to(device)
        class_encodings = torch.index_select(x, 0, indexes_tensor)
        stor = []
        for i in range(class_encodings.shape[0]):
            stor.append(dist_metric(class_encodings, class_encodings[i, :], c, norm_dist=norm_dist).sum())   #finds distance from all 
        ind = stor.index(min(stor))
        prototype = class_encodings[ind, :]
        prototypes.append(prototype)
    prototypes = torch.stack(prototypes, 0)
    return prototypes, unique_labels

In [13]:
def MMD_Prototype(x, y, c=2, norm_dist=False):
    '''
    indexes is a list of all encodings in x, where x has label 'unique_label'
    class_encodings further selects all such encodings from x from based on indexes from y
    class_encodings:  x11 x22 ...............x1d
                      x21 x22 ...............x2d
                      .                        .
                      .                        .
                      .                        .
                      xn1 xn2 ...............xnd
    prototype : x1 x2 x3 .................xd
    
    '''
    eps = 1e-7
    unique_labels = torch.unique(y)
    prototypes = []
    for unique_label in unique_labels:
        indexes = [i for i in range(y.shape[0]) if y[i]==unique_label]
        indexes_tensor = torch.tensor(indexes).to(device)
        class_encodings = torch.index_select(x, 0, indexes_tensor)
        IFs = []
        prototype = torch.mean(class_encodings,0)
        with torch.no_grad():
            for i in range(class_encodings.shape[0]):
                indexes = [ j for j in range(class_encodings.shape[0]) if j is not i]
                indexes_tensor = torch.tensor(indexes).to(device)
                exc_class_encodings = torch.index_select(class_encodings, 0, indexes_tensor)
                exc_prototype = torch.mean(exc_class_encodings, 0)
                MMD = (prototype - exc_prototype).square().sum()
                IF = torch.abs(1 - MMD)
                IFs.append(IF)
            IFs = torch.stack(IFs, 0).detach()
            IFs = F.normalize(IFs, p=1, dim=0)
            prototype = (IFs@class_encodings)/(IFs.sum()+eps)
            prototypes.append(prototype)
    prototypes = torch.stack(prototypes, 0)
    return prototypes, unique_labels

In [14]:
def Prototype(x, y, c=2, norm_dist=False):
    '''
    indexes is a list of all encodings in x, where x has label 'unique_label'
    class_encodings further selects all such encodings from x from based on indexes from y
    class_encodings:  x11 x22 ...............x1d
                      x21 x22 ...............x2d
                      .                        .
                      .                        .
                      .                        .
                      xn1 xn2 ...............xnd
    prototype : x1 x2 x3 .................xd
    
    '''
    unique_labels = torch.unique(y)
    prototypes = []
    for unique_label in unique_labels:
        indexes = [i for i in range(y.shape[0]) if y[i]==unique_label]
        indexes_tensor = torch.tensor(indexes).to(device)
        class_encodings = torch.index_select(x, 0, indexes_tensor)
        prototype = torch.mean(class_encodings, 0)
        prototypes.append(prototype)
    prototypes = torch.stack(prototypes, 0)
    return prototypes, unique_labels

In [15]:
def Get_Loss(prototypes, prototype_labels, eq, yq, es, ys, c=2, norm_dist=False, lam=0):
    '''
    prototypes : (Nc, XX), 
    prototype_labels : (Nc),
    eq : (Nq*Nc, XX),
    yq : (Nq*Nc)
    proto_eqs_dist : (Nq*Nc)
    protos_eqs_dist : (Nc, Nq*Nc)
    dist_sums : (Nq*Nc)
    '''
    eps = 1e-12
    protos_eqs_dist = torch.empty(size=(prototypes.shape[0], eq.shape[0])).to(device)
    protos_protos_dist = torch.empty(size=(prototypes.shape[0], prototypes.shape[0])).to(device)
    proto_dists = 0
    cls_protos_dist = 0
    for i in range(prototypes.shape[0]):
        prototype = prototypes[i, :]
        inds = [j for j in range(prototypes.shape[0]) if j!=i]
        cls_inds = [j for j in range(yq.shape[0]) if yq[j]==prototype_labels[i]]
        cls_inds_es = [j for j in range(ys.shape[0]) if ys[j]==prototype_labels[i]]
        proto_eqs_dist = torch.exp(-1*dist_metric(eq, prototype, c, norm_dist=norm_dist))
        proto_protos_dist = torch.exp(-1*dist_metric(prototypes[inds, :], prototype, c, norm_dist=norm_dist))
        proto_ess_dist = torch.exp(-1*dist_metric(es[cls_inds_es], prototype, c, norm_dist=norm_dist))
        protos_eqs_dist[i, :] = proto_eqs_dist
        val = proto_protos_dist.mean()/(proto_ess_dist.sum()+proto_eqs_dist[cls_inds].sum())
        proto_dists += val
    dist_sums = protos_eqs_dist.sum(0)
    loss = 0
    for i in range(prototype_labels.shape[0]):
        label = prototype_labels[i]
        indexes = [j for j in range(yq.shape[0]) if yq[j]==label]
        loss += -1*torch.log(torch.div(protos_eqs_dist[i, indexes], dist_sums[indexes]+eps)).sum()
    loss += lam*proto_dists.sum()
    # finding accuracy
    correct = 0
    total = yq.shape[0]
    val, ind = torch.max(protos_eqs_dist, 0)
    for i in range(yq.shape[0]):
        out_label = prototype_labels[ind[i]]
        true_label = yq[i]
        if true_label==out_label:
            correct+=1
    acc = float(correct)/total
    return loss/yq.shape[0], acc

In [16]:
def gaussian_kernel(x, y, sigma=1):
    # Calculate squared Euclidean distance between each pair of samples in x and y
    distances = torch.sum((x[:, None, :] - y[None, :, :]) ** 2, dim=-1)

    # Calculate kernel values using the squared distances and sigma
    kernel = torch.exp(-distances / (2 * sigma ** 2))

    return kernel

def Get_DA(eq, yq, es, ys, ev, yv, sigma):
    '''
    eq : (Nq*Nc, XX)
    es : (Ns*Nc, XX)
    ev : (Nv*Valid_split, XX)
    '''
    esq = torch.cat([eq, es], 0)
    DA1 = torch.sum(gaussian_kernel(esq, esq, sigma))/(esq.shape[0]**2)
    DA2 = torch.sum(gaussian_kernel(ev, ev, sigma))/(ev.shape[0]**2)
    DA3 = torch.sum(gaussian_kernel(esq, ev, sigma))/(esq.shape[0]*ev.shape[0])
    return DA1+DA2-2*DA3

# Train

In [17]:
def batch_train(encoder, 
                optimizer, 
                train_data, 
                test_data, 
                epochs, 
                batch_size, 
                c=2, 
                do_test=True, 
                schedule=None, 
                print_progress=True, 
                prototype_type='None', 
                norm_dist=False, 
                lam=0):
    '''
    XQ : (batch_size*Nq*Nc, Channels=3, W, H)  YQ: (batch_size, Nq*Nc)
    XS: (batch_size*Ns*Nc, Channels=3, W, H)   YS: (batch_size, Ns*Nc)
    EQ : (batch_size,Nq*Nc, XX)
    ES: (batch_size,Ns*Nc, XX)
    eq : (Nq*Nc, XX)
    es: (Ns*Nc, XX)
    yq : (Nq*Nc)
    ys : (Ns*Nc)
    prototypes : (Nc, XX), 
    prototype_labels : (Nc)
    '''
    train_loss_stor=[]
    train_acc_stor = []
    test_acc_stor = []
    test_loss_stor = []
    avg_test_loss = []
    avg_test_acc = []
    acc = 0
    loss = 0
    test_loss = 0
    test_acc = 0
    
    for ep, (XQ, YQ, XS, YS) in enumerate(tqdm(train_data)):
        encoder.train()
        xqdim=XQ.shape
        xsdim=XS.shape
        XQ=XQ.reshape(xqdim[0]*xqdim[1],xqdim[2],xqdim[3],xqdim[4]).to(device) 
        XS=XS.reshape(xsdim[0]*xsdim[1],xsdim[2],xsdim[3],xsdim[4]).to(device)
        optimizer.zero_grad()
        for epoch in range(epochs): 
            EQ = encoder(XQ) 
            ES = encoder(XS)
            EQ=EQ.reshape(xqdim[0],xqdim[1],-1).to(device) 
            ES=ES.reshape(xsdim[0],xsdim[1],-1).to(device)
            running_loss=0
            running_acc = 0
            for batch in range(batch_size):
                eq=EQ[batch,:,:]
                es=ES[batch,:,:]
                ys=YS[batch,:]
                yq=YQ[batch,:]
                if prototype_type=="cluster":
                    prototypes, prototype_labels = cluster_Prototype(es, ys, c, norm_dist=norm_dist) 
                elif prototype_type=='MMD':
                    prototypes, prototype_labels = MMD_Prototype(es, ys, c, norm_dist=norm_dist)
                else:
                    prototypes, prototype_labels = Prototype(es, ys, c, norm_dist=norm_dist)
#                 print(prototypes)
                batch_loss, batch_acc = Get_Loss(prototypes, prototype_labels, eq, yq, es, ys, c=c, norm_dist=norm_dist, lam=lam)
#                 print('LOSS ', batch_loss)
                running_loss+=batch_loss/batch_size
                running_acc += batch_acc/batch_size
#                 break
            wandb.log({"train_loss_"+encoder.name: running_loss})
            wandb.log({"train_acc_"+encoder.name: running_acc})
            running_loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            running_acc = running_acc
            running_loss = running_loss.item()
            train_loss_stor.append(running_loss)
            train_acc_stor.append(running_acc)
#         break
    #         print(acc)
    #         print(prototypes.shape, prototype_labels, 'PROTOTYPE')
    #         print(epoch, xq.shape, yq.shape, "SHAPE Check")
    #         print(eq.shape, es.shape, 'ENCODE SHAPE')
        if schedule:
            schedule.step()
    #     optimizer.step()
        encoder.eval()
        avg_loss = torch.tensor(0).to(device)
        avg_acc = 0
        if do_test:
            with torch.no_grad():
                for ep_test, (XQ, YQ, XS, YS) in enumerate(test_data):
                    xqdim=XQ.shape
                    xsdim=XS.shape
                    XQ=XQ.reshape(xqdim[0]*xqdim[1],xqdim[2],xqdim[3],xqdim[4])
                    XS=XS.reshape(xsdim[0]*xsdim[1],xsdim[2],xsdim[3],xsdim[4])
                    EQ = encoder(XQ) 
                    ES = encoder(XS) 
                    EQ=EQ.reshape(xqdim[0],xqdim[1],-1)
                    ES=ES.reshape(xsdim[0],xsdim[1],-1)
                    running_test_loss=0
                    running_test_acc = 0
                    for batch in range(batch_size):
                        eq=EQ[batch,:,:]
                        es=ES[batch,:,:]
                        ys=YS[batch,:]
                        yq=YQ[batch,:]
                        if prototype_type=="cluster":
                            prototypes, prototype_labels = cluster_Prototype(es, ys, c, norm_dist=norm_dist) 
                        elif prototype_type=='MMD':
                            prototypes, prototype_labels = MMD_Prototype(es, ys, c, norm_dist=norm_dist)
                        else:
                            prototypes, prototype_labels = Prototype(es, ys, c, norm_dist=norm_dist)
                        batch_loss, batch_acc = Get_Loss(prototypes, prototype_labels, eq, yq, es, ys, c=c, norm_dist=norm_dist, lam=lam)
                        running_test_loss+=batch_loss/batch_size
                        running_test_acc += batch_acc/batch_size
                avg_loss = running_test_loss
                avg_acc = running_test_acc
                wandb.log({"test_loss_"+encoder.name: avg_loss.item()})
                wandb.log({"test_acc_"+encoder.name: avg_acc})
                avg_test_loss.append(avg_loss.item())
                avg_test_acc.append(avg_acc)
        if print_progress:
            print(f'Encoder:{encoder.name} Epoch:{ep}/{config.train_enum / batch_size} Train Acc: {round(running_acc,3)}, Test Acc: {round(avg_acc,3)}, Train_loss: {round(running_loss,3)}, Test_loss: {round(avg_loss.item(),3)} ', end='\r')
    
    print(f'Encoder:{encoder.name} Epoch:{ep}/{config.train_enum / batch_size} Train Acc: {round(running_acc,3)}, Test Acc {round(avg_acc,3)}, Train_loss: {round(running_loss,3)}, Test_loss: {round(avg_loss.item(),3)}')
    
    test_loss_stor = avg_test_loss
    test_acc_stor = avg_test_acc
    return {"train_loss_stor": train_loss_stor,
            "train_acc_stor": train_acc_stor,
            "avg_test_loss": avg_test_loss,
            "avg_test_acc": avg_test_acc
           }


In [18]:
def batch_train_DAPN(encoder, 
                optimizer, 
                train_data, 
                test_data, 
                epochs, 
                batch_size, 
                c=2, 
                do_test=True, 
                schedule=None, 
                print_progress=True, 
                prototype_type='None', 
                norm_dist=False, 
                lam=0,
                alpha=0.2,
                sigma=1):
    '''
    XQ : (batch_size*Nq*Nc, Channels=3, W, H)  YQ: (batch_size, Nq*Nc)
    XS: (batch_size*Ns*Nc, Channels=3, W, H)   YS: (batch_size, Ns*Nc)
    EQ : (batch_size,Nq*Nc, XX)
    ES: (batch_size,Ns*Nc, XX)
    eq : (Nq*Nc, XX)
    es: (Ns*Nc, XX)
    yq : (Nq*Nc)
    ys : (Ns*Nc)
    prototypes : (Nc, XX), 
    prototype_labels : (Nc)
    '''
    train_loss_stor=[]
    train_acc_stor = []
    test_acc_stor = []
    test_loss_stor = []
    avg_test_loss = []
    avg_test_acc = []
    acc = 0
    loss = 0
    test_loss = 0
    test_acc = 0
    
    for ep, (XQ, YQ, XS, YS, XV, YV) in enumerate(tqdm(train_data)):
        encoder.train()
        xqdim=XQ.shape
        xsdim=XS.shape
        xvdim=XV.shape
        XQ=XQ.reshape(xqdim[0]*xqdim[1],xqdim[2],xqdim[3],xqdim[4]).to(device) 
        XS=XS.reshape(xsdim[0]*xsdim[1],xsdim[2],xsdim[3],xsdim[4]).to(device)
        XV=XV.reshape(xvdim[0]*xvdim[1],xvdim[2],xvdim[3],xvdim[4]).to(device)
        optimizer.zero_grad()
        for epoch in range(epochs): 
            EQ = encoder(XQ) 
            ES = encoder(XS)
            EV = encoder(XV)
            EQ=EQ.reshape(xqdim[0],xqdim[1],-1).to(device) 
            ES=ES.reshape(xsdim[0],xsdim[1],-1).to(device)
            EV=EV.reshape(xvdim[0],xvdim[1],-1).to(device)
            running_loss=0
            running_acc = 0
            for batch in range(batch_size):
                eq=EQ[batch,:,:]
                es=ES[batch,:,:]
                ev=EV[batch,:,:]
                ys=YS[batch,:]
                yq=YQ[batch,:]
                yv=YV[batch,:]
                if prototype_type=="cluster":
                    prototypes, prototype_labels = cluster_Prototype(es, ys, c, norm_dist=norm_dist) 
                elif prototype_type=='MMD':
                    prototypes, prototype_labels = MMD_Prototype(es, ys, c, norm_dist=norm_dist)
                else:
                    prototypes, prototype_labels = Prototype(es, ys, c, norm_dist=norm_dist)
#                 print(prototypes)
                batch_loss, batch_acc = Get_Loss(prototypes, prototype_labels, eq, yq, es, ys, c=c, norm_dist=norm_dist, lam=lam)
                da_loss = Get_DA(eq, yq, es, ys, ev, yv, sigma=sigma)
#                 print("DAPN Comparison", batch_loss, da_loss)
                batch_loss +=alpha*da_loss
#                 print('LOSS ', batch_loss)
                running_loss+=batch_loss/batch_size
                running_acc += batch_acc/batch_size
#                 break
            wandb.log({"train_loss_"+encoder.name: running_loss})
            wandb.log({"train_acc_"+encoder.name: running_acc})
            running_loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            running_acc = running_acc
            running_loss = running_loss.item()
            train_loss_stor.append(running_loss)
            train_acc_stor.append(running_acc)
#         break
    #         print(acc)
    #         print(prototypes.shape, prototype_labels, 'PROTOTYPE')
    #         print(epoch, xq.shape, yq.shape, "SHAPE Check")
    #         print(eq.shape, es.shape, 'ENCODE SHAPE')
        if schedule:
            schedule.step()
    #     optimizer.step()
        encoder.eval()
        avg_loss = torch.tensor(0).to(device)
        avg_acc = 0
        if do_test:
            with torch.no_grad():
                for ep_test, (XQ, YQ, XS, YS) in enumerate(test_data):
                    xqdim=XQ.shape
                    xsdim=XS.shape
                    XQ=XQ.reshape(xqdim[0]*xqdim[1],xqdim[2],xqdim[3],xqdim[4])
                    XS=XS.reshape(xsdim[0]*xsdim[1],xsdim[2],xsdim[3],xsdim[4])
                    EQ = encoder(XQ) 
                    ES = encoder(XS) 
                    EQ=EQ.reshape(xqdim[0],xqdim[1],-1)
                    ES=ES.reshape(xsdim[0],xsdim[1],-1)
                    running_test_loss=0
                    running_test_acc = 0
                    for batch in range(batch_size):
                        eq=EQ[batch,:,:]
                        es=ES[batch,:,:]
                        ys=YS[batch,:]
                        yq=YQ[batch,:]
                        if prototype_type=="cluster":
                            prototypes, prototype_labels = cluster_Prototype(es, ys, c, norm_dist=norm_dist) 
                        elif prototype_type=='MMD':
                            prototypes, prototype_labels = MMD_Prototype(es, ys, c, norm_dist=norm_dist)
                        else:
                            prototypes, prototype_labels = Prototype(es, ys, c, norm_dist=norm_dist)
                        batch_loss, batch_acc = Get_Loss(prototypes, prototype_labels, eq, yq, es, ys, c=c, norm_dist=norm_dist, lam=lam)
                        running_test_loss+=batch_loss/batch_size
                        running_test_acc += batch_acc/batch_size
                avg_loss = running_test_loss
                avg_acc = running_test_acc
                wandb.log({"test_loss_"+encoder.name: avg_loss.item()})
                wandb.log({"test_acc_"+encoder.name: avg_acc})
                avg_test_loss.append(avg_loss.item())
                avg_test_acc.append(avg_acc)
        if print_progress:
            print(f'Encoder:{encoder.name} Epoch:{ep}/{config.train_enum / batch_size} Train Acc: {round(running_acc,3)}, Test Acc: {round(avg_acc,3)}, Train_loss: {round(running_loss,3)}, Test_loss: {round(avg_loss.item(),3)} ', end='\r')
    
    print(f'Encoder:{encoder.name} Epoch:{ep}/{config.train_enum / batch_size} Train Acc: {round(running_acc,3)}, Test Acc {round(avg_acc,3)}, Train_loss: {round(running_loss,3)}, Test_loss: {round(avg_loss.item(),3)}')
    
    test_loss_stor = avg_test_loss
    test_acc_stor = avg_test_acc
    return {"train_loss_stor": train_loss_stor,
            "train_acc_stor": train_acc_stor,
            "avg_test_loss": avg_test_loss,
            "avg_test_acc": avg_test_acc
           }


# Evaluation

In [19]:
def batch_evaluate_model(encoder, dataload, batch_size, prototype_type, plot, c=2, norm_dist=False, lam=0):
    '''
    XQ : (batch_size*Nq*Nc, Channels=3, W, H)  YQ: (batch_size, Nq*Nc)
    XS: (batch_size*Ns*Nc, Channels=3, W, H)   YS: (batch_size, Ns*Nc)
    EQ : (batch_size,Nq*Nc, XX)
    ES: (batch_size,Ns*Nc, XX)
    eq : (Nq*Nc, XX)
    es: (Ns*Nc, XX)
    yq : (Nq*Nc)
    ys : (Ns*Nc)
    prototypes : (Nc, XX), 
    prototype_labels : (Nc)
    '''
    encoder.eval()
    avg_loss = 0
    avg_acc = 0
    main_test_loss = []
    main_test_acc = []
    with torch.no_grad():
        for ep_test, (XQ, YQ, XS, YS) in enumerate(dataload):
            xqdim=XQ.shape
            xsdim=XS.shape
            XQ=XQ.reshape(xqdim[0]*xqdim[1],xqdim[2],xqdim[3],xqdim[4])# 
            XS=XS.reshape(xsdim[0]*xsdim[1],xsdim[2],xsdim[3],xsdim[4])# 

            EQ = encoder(XQ)  #  
            ES = encoder(XS) #  
            EQ=EQ.reshape(xqdim[0],xqdim[1],-1) #  
            ES=ES.reshape(xsdim[0],xsdim[1],-1)#  
            running_test_loss=0
            running_test_acc = 0
            for batch in range(batch_size):
                eq=EQ[batch,:,:]
                es=ES[batch,:,:]
                ys=YS[batch,:]
                yq=YQ[batch,:]
                if prototype_type=="cluster":
                    prototypes, prototype_labels = cluster_Prototype(es, ys, c, norm_dist=norm_dist) 
                elif prototype_type=='MMD':
                    prototypes, prototype_labels = MMD_Prototype(es, ys, c, norm_dist=norm_dist)
                else:
                    prototypes, prototype_labels = Prototype(es, ys, c, norm_dist=norm_dist)
                batch_loss, batch_acc = Get_Loss(prototypes, prototype_labels, eq, yq, es, ys, c=c, norm_dist=norm_dist, lam=lam)
                running_test_loss+=batch_loss/batch_size
                running_test_acc += batch_acc/batch_size
            avg_loss = running_test_loss
            avg_acc = running_test_acc
            main_test_loss.append(avg_loss.item())
            main_test_acc.append(avg_acc)
    Nmain_test_loss = np.array(main_test_loss)
    Nmain_test_acc = np.array(main_test_acc)
    print(f'Loss: Mean:--{Nmain_test_loss.mean()}  Variance:--{Nmain_test_loss.var()}')
    print(f'Accuracy: Mean:--{Nmain_test_acc.mean()}  Variance:--{Nmain_test_acc.var()}')
    if plot:
        plt.plot(main_test_loss)
        plt.show()
        plt.plot(main_test_acc)
        plt.show()
    return Nmain_test_acc.mean(), Nmain_test_loss.mean()

In [20]:
def plot_results(result_array, title="None"):
    print(title)
    plt.plot(result_array)
    plt.show()
    result_np = np.array(result_array)
    print("Mean: ", result_np[:int(len(result_array)*0.9)].mean())
    print("Variance: ", result_np.var())

# Result

In [21]:
conf = dict(
    # For Training Data Parameters
    train_way =  5,
    train_shot =  5,
    train_query = 5,
    # For Testing Data Parameters
    test_way = 5,
    test_shot = 5,
    test_query = 5,
    # For during Training and Testing Parameters
    Epochs = 1,              # number of times a DataPoint is trained
    train_enum = 1000,        # Number of datapoints picked
    test_enum = 5,           # Number of test points picked per epoch
    test_fin = 400,          # number of test points picked at end of train
    # For Training HyperParameters
    l_r = 1e-3, 
    embedding_space=64,
    keep_momentum = True,     
    keep_schedule = True,    # keep learning rate scheduling (outer loop)
    schedule_stepsize = 100,  # step size for learning rate reducing (outer loop)
    gamma = 0.5,             # gamma for learning rate reducing
    momentum = 0.9, 
    batch_size = 5,
    train_block = 6,         # number of training blocks
    cdist = 2,             # type of norm
    prototype_type="None",            # form clusters or take mean
    norm_dist = 64,          # normalize the distances
    lam = 0,                # cluster seperation factor
    # Image parameters
    reshape = True,       
    Width = 120,
    Height =  120,
    norm = 255,              # normalize images
    #Data Paths
    single_data = 0,
    #DAPN-parameter
    alpha=1,
    sigma=3
)


In [22]:
config = dict2obj(conf)

init = wandb.init(project="prototypical_network_second", 
           entity="team_prototype", 
           config=conf, 
           notes= 'Original Model', 
#            name=f'lam{config.lam}-{config.test_way}w{config.test_shot}s-b{config.batch_size}',
           name=f'6enc--{config.alpha}-{config.sigma}DAPN-{config.test_way}w{config.test_shot}s',
#            mode="disabled"
          )
# c{config.cdist}-lam{config.lam}-
# creating medoid cluster and adding lambda
# Orig-{config.train_block}conv{config.test_way}

wandb: Currently logged in as: team_prototype (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.15.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [23]:
train_paths = [
    "../input/derm7pt-augmented/Train_split_aug-20220217T151629Z-001/Train_split_aug/Train/",  #DERM
    "../input/miniimagenet/data/miniImageNet/images_background/",                              #MiniImage
    "../input/omniglot/images_background/",                                                    #OmmniGlot
    '../input/derm7pt-fold/Data_derm/Fold_',                                                   #Initials  
    '/train_max_aug/',
    '../input/derm7pt-50-aug/50_aug_Fold/Fold_'
]

test_paths = [
    "../input/derm7pt-augmented/Test-20220217T151622Z-001/Test/",  #DERM
    "../input/miniimagenet/data/miniImageNet/images_evaluation/",  #MiniImage
    "../input/omniglot/images_evaluation/",                        #OmmniGlot
    '../input/derm7pt-fold/Data_derm/Fold_',                       #Initials
    '/Test1/',                                                     #folds
    '../input/derm7pt-50-aug/50_aug_Fold/Fold_'
]

# MODEL RUN FUNCTIONS

In [24]:
def Apply_Model(test_path, 
                train_path,  
                prototype_type, 
                deep=False, 
                do_test=True, 
                print_progress=True, 
                print_eval=False, 
                plot_result=False, 
                name='0', 
                norm_dist=False,
               train_eval=True):
    print("************************READING************************")
    if deep:
        test_data = data_preload_deep(test_path, [config.Height, config.Width], norm=config.norm, reshape=config.reshape)
        train_data = data_preload_deep(train_path, [config.Height, config.Width], norm=config.norm, reshape=config.reshape)
    else:
        test_data = data_preload(test_path, [config.Height, config.Width], norm=config.norm, reshape=config.reshape)
        train_data = data_preload(train_path, [config.Height, config.Width], norm=config.norm, reshape=config.reshape)
#     print('test data: ', test_data.shape)
    print_data(test_data, 'test data')
    print_data(train_data, 'train data')
#     print('train data: ', train_data.shape)
    train_dataset = ProtoDataset(train_data, Nc=config.train_way, Ns=config.train_shot, Nq=config.train_query, epoch=config.train_enum)
    train_dataload = DataLoader(dataset=train_dataset, 
                                batch_size=config.batch_size
                               )
    test_dataset = ProtoDataset(test_data, Nc=config.test_way, Ns=config.test_shot, Nq=config.test_query, epoch=config.test_enum)
    test_dataload = DataLoader(dataset=test_dataset,
                               batch_size=config.batch_size
                              )
    eval_dataset = ProtoDataset(test_data, Nc=config.test_way, Ns=config.test_shot, Nq=config.test_query, epoch=config.test_fin)
    eval_dataload = DataLoader(dataset=eval_dataset,
                              batch_size=config.batch_size
                              )
    traineval_dataset = ProtoDataset(train_data, Nc=config.test_way, Ns=config.test_shot, Nq=config.test_query, epoch=config.test_fin)
    traineval_dataload = DataLoader(dataset=traineval_dataset,
                                  batch_size=config.batch_size
                                  )
    print("************************TRAINING************************")
    encoder = Encoder(config.embedding_space, config.train_block, name, drop=False).to(device)
    if config.keep_momentum:
        optimizer = optim.Adam(encoder.parameters(), lr=config.l_r,
#                                momentum=config.momentum
                              )
    else:
        optimizer = optim.Adam(encoder.parameters(), lr=config.l_r)
    if config.keep_schedule:
        scheduler = optim.lr_scheduler.StepLR(optimizer, config.schedule_stepsize, gamma=config.gamma, last_epoch=-1)
    else:
        scheduler=None
    wandb.watch(encoder)
    results = batch_train(encoder,
                      optimizer,
                      train_dataload,
                      test_dataload,
                      config.Epochs,
                      batch_size=config.batch_size,
                      do_test=do_test,
                      schedule=scheduler,
                      print_progress=print_progress,
                      c=config.cdist,
                      prototype_type=prototype_type,
                      norm_dist=norm_dist,
                      lam=config.lam
                     )
    if plot_result:
        for result in results:
            plot_results(results[result], result)
    print("************************TestEVAL************************")
    te_acc, te_loss = batch_evaluate_model(encoder, 
                                           eval_dataload, 
                                           config.batch_size, 
                                           plot=print_eval, 
                                           c=config.cdist, 
                                           prototype_type=prototype_type, 
                                           norm_dist=config.norm_dist,
                                           lam=config.lam)
    wandb.log({
        'test_acc_eval_'+encoder.name: te_acc,
        'test_loss_eval_'+encoder.name: te_loss
    })
    tr_acc = 0
    tr_loss = 0
    if train_eval:
        print("************************TrainEVAL************************")
        tr_acc, tr_loss = batch_evaluate_model(encoder, 
                                               traineval_dataload,
                                               config.batch_size, 
                                               plot=print_eval, 
                                               c=config.cdist, 
                                               prototype_type=prototype_type, 
                                               norm_dist=config.norm_dist,
                                               lam=config.lam)
        wandb.log({
            'train_acc_eval_'+encoder.name: tr_acc,
            'train_loss_eval_'+encoder.name: tr_loss,
        })
    return results, te_acc, te_loss, tr_acc, tr_loss


In [25]:
def Apply_DAPN(test_path, 
                train_path,  
                prototype_type, 
                deep=False, 
                do_test=True, 
                print_progress=True, 
                print_eval=False, 
                plot_result=False, 
                name='0', 
                norm_dist=False,
               train_eval=True):
    print("************************READING************************")
    if deep:
        test_data = data_preload_deep(test_path, [config.Height, config.Width], norm=config.norm, reshape=config.reshape)
        train_data = data_preload_deep(train_path, [config.Height, config.Width], norm=config.norm, reshape=config.reshape)
    else:
        test_data = data_preload(test_path, [config.Height, config.Width], norm=config.norm, reshape=config.reshape)
        train_data = data_preload(train_path, [config.Height, config.Width], norm=config.norm, reshape=config.reshape)
#     print('test data: ', test_data.shape)
    print_data(test_data, 'test data')
    print_data(train_data, 'train data')
#     print('train data: ', train_data.shape)
    train_dataset = DAPNDataset(train_data, Nc=config.train_way, Ns=config.train_shot, Nq=config.train_query,Nv=4 ,valid_split=3 ,epoch=config.train_enum)
    train_dataload = DataLoader(dataset=train_dataset, 
                                batch_size=config.batch_size
                               )
    test_dataset = ProtoDataset(test_data, Nc=config.test_way, Ns=config.test_shot, Nq=config.test_query, epoch=config.test_enum)
    test_dataload = DataLoader(dataset=test_dataset,
                               batch_size=config.batch_size
                              )
    eval_dataset = ProtoDataset(test_data, Nc=config.test_way, Ns=config.test_shot, Nq=config.test_query, epoch=config.test_fin)
    eval_dataload = DataLoader(dataset=eval_dataset,
                              batch_size=config.batch_size
                              )
    traineval_dataset = ProtoDataset(train_data, Nc=config.test_way, Ns=config.test_shot, Nq=config.test_query, epoch=config.test_fin)
    traineval_dataload = DataLoader(dataset=traineval_dataset,
                                  batch_size=config.batch_size
                                  )
    print("************************TRAINING************************")
    encoder = Encoder(config.embedding_space, config.train_block, name, drop=False).to(device)
    if config.keep_momentum:
        optimizer = optim.Adam(encoder.parameters(), lr=config.l_r,
#                                momentum=config.momentum
                              )
    else:
        optimizer = optim.Adam(encoder.parameters(), lr=config.l_r)
    if config.keep_schedule:
        scheduler = optim.lr_scheduler.StepLR(optimizer, config.schedule_stepsize, gamma=config.gamma, last_epoch=-1)
    else:
        scheduler=None
    wandb.watch(encoder)
    results = batch_train_DAPN(encoder,
                      optimizer,
                      train_dataload,
                      test_dataload,
                      config.Epochs,
                      batch_size=config.batch_size,
                      do_test=do_test,
                      schedule=scheduler,
                      print_progress=print_progress,
                      c=config.cdist,
                      prototype_type=prototype_type,
                      norm_dist=norm_dist,
                      lam=config.lam,
                      alpha=config.alpha,
                      sigma=config.sigma
                     )
    if plot_result:
        for result in results:
            plot_results(results[result], result)
    print("************************TestEVAL************************")
    te_acc, te_loss = batch_evaluate_model(encoder, 
                                           eval_dataload, 
                                           config.batch_size, 
                                           plot=print_eval, 
                                           c=config.cdist, 
                                           prototype_type=prototype_type, 
                                           norm_dist=config.norm_dist,
                                           lam=config.lam)
    wandb.log({
        'test_acc_eval_'+encoder.name: te_acc,
        'test_loss_eval_'+encoder.name: te_loss
    })
    tr_acc = 0
    tr_loss = 0
    if train_eval:
        print("************************TrainEVAL************************")
        tr_acc, tr_loss = batch_evaluate_model(encoder, 
                                               traineval_dataload,
                                               config.batch_size, 
                                               plot=print_eval, 
                                               c=config.cdist, 
                                               prototype_type=prototype_type, 
                                               norm_dist=config.norm_dist,
                                               lam=config.lam)
        wandb.log({
            'train_acc_eval_'+encoder.name: tr_acc,
            'train_loss_eval_'+encoder.name: tr_loss,
        })
    return results, te_acc, te_loss, tr_acc, tr_loss


In [26]:
def Kfold_batch_apply(folds=[1, 2, 3, 4, 5], do_test=True,  print_progress=True, train_eval=True):
    '''
    Check whether Apply_Model is called or Apply_DAPN is called
    '''
    test_acc = []
    test_loss = []
    train_acc = []
    train_loss = []
    for i in folds:
        test_path = test_paths[3] +str(i)+test_paths[4]                       #test_paths[config.single_data]
        train_path = train_paths[3]+str(i)+train_paths[4]                       #train_paths[config.single_data]
        results, te_acc, te_loss, tr_acc, tr_loss = Apply_DAPN(test_path=test_path, 
                                                                train_path=train_path, 
                                                                name=str(i), 
                                                                prototype_type=config.prototype_type, 
                                                                norm_dist=config.norm_dist,
                                                                do_test=do_test,
                                                               train_eval = train_eval)
        test_acc.append(te_acc)
        test_loss.append(te_loss)
        train_acc.append(tr_acc)
        train_loss.append(tr_loss) 
    test_acc = np.array(test_acc)
    test_loss = np.array(test_loss)
    train_acc = np.array(train_acc)
    train_loss = np.array(train_loss)
    print(f'TEST KFOLDS : Mean Acc: {test_acc.mean()} Variance Acc: {test_acc.var()} Mean Loss: {test_loss.mean()}  Variance Loss: {test_loss.var()}')
    print(f'TRAIN KFOLDS: Mean Acc: {train_acc.mean()} Variance Acc: {train_acc.var()} Mean Loss: {train_loss.mean()}  Variance Loss: {train_loss.var()}')
    wandb.log({
        'test_mean_acc': test_acc.mean(),
        'test_var_acc': test_acc.var(),
        'test_mean_loss': test_loss.mean(),
        'test_var_loss': test_loss.var(),
        'train_mean_acc': train_acc.mean(),
        'train_var_acc': train_acc.var(),
        'train_mean_loss': train_loss.mean(),
        'train_var_loss': train_loss.var(),
    })
    return test_acc, test_loss, train_acc, train_loss

In [27]:
# Apply_model(test_path=test_paths[config.single_data], 
#             train_path=train_paths[config.single_data], 
#             deep=False, 
#             do_test=False, 
#             print_progress=True, 
#             print_eval=False, 
#             plot_result=True
#            )

In [28]:
print(f'{config.test_way}w{config.test_shot}s')
Kfold_batch_apply(folds=[
                        1, 
                        2, 
                        3, 
                        4, 
                        5
                        ],
                 do_test=False,
                 train_eval=False)

5w5s
************************READING************************
test data : (12, 3, 120, 120) (13, 3, 120, 120) (12, 3, 120, 120) (14, 3, 120, 120) (13, 3, 120, 120) (12, 3, 120, 120) 

train data : (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) 

************************TRAINING************************


  0%|          | 0/200 [00:00<?, ?it/s]

Encoder:1 Epoch:199/200.0 Train Acc: 0.984, Test Acc 0, Train_loss: 0.535, Test_loss: 0
************************TestEVAL************************
Loss: Mean:--1.276397594809532  Variance:--0.0003146369217328892
Accuracy: Mean:--0.47780000000000006  Variance:--0.0019807600000000002
************************READING************************
test data : (13, 3, 120, 120) (11, 3, 120, 120) (13, 3, 120, 120) (11, 3, 120, 120) (14, 3, 120, 120) (12, 3, 120, 120) 

train data : (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) 

************************TRAINING************************


  0%|          | 0/200 [00:00<?, ?it/s]

Encoder:2 Epoch:199/200.0 Train Acc: 0.976, Test Acc 0, Train_loss: 0.585, Test_loss: 0
************************TestEVAL************************
Loss: Mean:--1.3325370982289315  Variance:--0.0006099705331231475
Accuracy: Mean:--0.40709999999999996  Variance:--0.00128479
************************READING************************
test data : (14, 3, 120, 120) (11, 3, 120, 120) (15, 3, 120, 120) (15, 3, 120, 120) (13, 3, 120, 120) 

train data : (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) 

************************TRAINING************************


...

  0%|          | 0/200 [00:00<?, ?it/s]

Encoder:4 Epoch:199/200.0 Train Acc: 0.976, Test Acc 0, Train_loss: 0.588, Test_loss: 0
************************TestEVAL************************
Loss: Mean:--1.1188207805156707  Variance:--0.003590586852594733
Accuracy: Mean:--0.5516  Variance:--0.00275504
************************READING************************
test data : (15, 3, 120, 120) (11, 3, 120, 120) (12, 3, 120, 120) (14, 3, 120, 120) (13, 3, 120, 120) (11, 3, 120, 120) 

train data : (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) (22, 3, 120, 120) 

************************TRAINING************************


  0%|          | 0/200 [00:00<?, ?it/s]

Encoder:5 Epoch:199/200.0 Train Acc: 0.936, Test Acc 0, Train_loss: 0.638, Test_loss: 0
************************TestEVAL************************
Loss: Mean:--1.4759930655360223  Variance:--0.001006519422982153
Accuracy: Mean:--0.3681  Variance:--0.0018823900000000005
TEST KFOLDS : Mean Acc: 0.47065999999999997 Variance Acc: 0.0054501864 Mean Loss: 1.2675066140294073  Variance Loss: 0.017552737287692605
TRAIN KFOLDS: Mean Acc: 0.0 Variance Acc: 0.0 Mean Loss: 0.0  Variance Loss: 0.0


(array([0.4778, 0.4071, 0.5487, 0.5516, 0.3681]),
 array([1.27639759, 1.3325371 , 1.13378453, 1.11882078, 1.47599307]),
 array([0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0]))

In [29]:
wandb.finish()

test_acc_eval_1,▁
test_acc_eval_2,▁
test_acc_eval_3,▁
test_acc_eval_4,▁
test_acc_eval_5,▁
test_loss_eval_1,▁
test_loss_eval_2,▁
test_loss_eval_3,▁
test_loss_eval_4,▁
test_loss_eval_5,▁
test_mean_acc,▁
